In [0]:
from pyspark.sql.functions import col, trim, lower, regexp_replace

# (double-check the table name; list to be sure)
spark.sql("SHOW TABLES IN default").show(truncate=False)

# load your table (use the exact one you created)
sent_df = spark.table("workspace.default.social_media_sentiment_target_80")

# clean text
sent_df = (sent_df
           .withColumn("tweet_text", trim(col("tweet_text")))
           .withColumn("username", trim(col("username")))
           .filter(col("tweet_text").isNotNull())
           .withColumn("tweet_text", lower(col("tweet_text")))
           .withColumn("tweet_text", regexp_replace(col("tweet_text"), r"[^a-z\s]", " ")))

print("Rows:", sent_df.count())
display(sent_df.groupBy("sentiment").count())


+--------+--------------------------------+-----------+
|database|tableName                       |isTemporary|
+--------+--------------------------------+-----------+
|default |social_media_sentiment          |false      |
|default |social_media_sentiment_92_acc   |false      |
|default |social_media_sentiment_highacc  |false      |
|default |social_media_sentiment_target_80|false      |
+--------+--------------------------------+-----------+

Rows: 500


sentiment count Neutral 141 Positive 193 Negative 166

In [0]:
train_raw, test_raw = sent_df.randomSplit([0.8, 0.2], seed=42)
print("Train:", train_raw.count(), "Test:", test_raw.count())


Train: 408 Test: 92


In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

# text pipeline
tokenizer = Tokenizer(inputCol="tweet_text", outputCol="words_raw")
stop_remover = StopWordsRemover(inputCol="words_raw", outputCol="words")
cv = CountVectorizer(inputCol="words", outputCol="raw_features", vocabSize=8000, minDF=1)
idf = IDF(inputCol="raw_features", outputCol="features")

text_pipe = Pipeline(stages=[tokenizer, stop_remover, cv, idf]).fit(train_raw)

# label indexer (fit on train)
labler = StringIndexer(inputCol="sentiment", outputCol="label").fit(train_raw)

# transform
train = labler.transform(text_pipe.transform(train_raw))
test  = labler.transform(text_pipe.transform(test_raw))

train.select("tweet_text","sentiment","label").show(5, truncate=False)


+---------------------------------------------+---------+-----+
|tweet_text                                   |sentiment|label|
+---------------------------------------------+---------+-----+
|good functional banking                      |Neutral  |2.0  |
|good experience but sometimes hardly loved   |Neutral  |2.0  |
|login feels loved though  crashes happens    |Positive |0.0  |
|poor support but sometimes hardly buggy      |Positive |0.0  |
|app feels crashes though  excellent happens  |Negative |1.0  |
+---------------------------------------------+---------+-----+
only showing top 5 rows


In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=100, regParam=0.01, elasticNetParam=0.0)
lr_model = lr.fit(train)

pred = lr_model.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
acc = evaluator.evaluate(pred)
print(f"Accuracy: {acc:.4f}")


Accuracy: 0.6304


In [0]:
from pyspark.ml.feature import IndexToString
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.sql.functions import udf

# map numeric prediction back to string
to_label = IndexToString(inputCol="prediction", outputCol="predicted_sentiment", labels=labler.labels)

# vector -> array
vec_to_arr = udf(lambda v: v.toArray().tolist(), ArrayType(DoubleType()))

pretty = to_label.transform(pred).withColumn("prob_array", vec_to_arr("probability"))
display(pretty.select("tweet_text","sentiment","predicted_sentiment","prob_array").limit(20))


tweet_text sentiment predicted_sentiment prob_array decent easy support Positive Positive List(0.6805236949001346, 0.1878837002197705, 0.1315926048800948) interface feels slow though barely secure happens Neutral Negative List(0.16860344158215576, 0.8178441881452888, 0.013552370272555592) great service but sometimes failed Negative Negative List(0.19756922014954523, 0.5787370815246742, 0.22369369832578065) transfer feels crashes though hardly secure happens Negative Positive List(0.8607281026720268, 0.1259656383075103, 0.013306259020462789) decent excellent login Positive Positive List(0.8480438457566005, 0.10667073556955517, 0.04528541867384439) failed interface is mixed Positive Negative List(0.17101305249798907, 0.7342049679482895, 0.09478197955372153) the experience is error and mixed Negative Negative List(0.13084066791848656, 0.7288846486148826, 0.14027468346663086) mostly terrible app is normal Negative Negative List(0.3212219460880766, 0.6586259326999512, 0.020152121211972397) the interface is buggy and mixed Negative Positive List(0.5308742099911222, 0.43622616460982244, 0.032899625399055395) the interface is functional and mixed Positive Neutral List(0.07223615189108866, 0.05880237032012877, 0.8689614777887825) crashes experience is mixed Negative Negative List(0.1606238150773761, 0.7606085012008355, 0.07876768372178852) good loved transfer Positive Positive List(0.7736184745432457, 0.1600486755705862, 0.06633284988616793) decent normal experience Neutral Neutral List(0.05255273264425242, 0.07458003422496726, 0.8728672331307804) good average feature Neutral Neutral List(0.10955239037566036, 0.07044948488204393, 0.8199981247422956) mixed service but sometimes error Positive Negative List(0.25708500551834607, 0.42066344966135616, 0.3222515448202978) great average support Neutral Neutral List(0.10560786669905686, 0.05057790545516882, 0.8438142278457744) mostly reliable transfer is normal Positive Positive List(0.8945706252550736, 0.07832087083953143, 0.0271085039053948) mixed fine app Neutral Neutral List(0.04017901543383143, 0.14112617726474472, 0.8186948073014237) mostly smooth upi is average Positive Positive List(0.7716404127053594, 0.1736608438653197, 0.054698743429320955) terrible upi but sometimes no longer buggy Negative Negative List(0.3380333424463378, 0.5798330297260069, 0.08213362782765525)

In [0]:
pdf = pretty.select("label","prediction").toPandas()
import pandas as pd
cm = pd.crosstab(pdf["label"], pdf["prediction"], rownames=["Actual"], colnames=["Predicted"], margins=True)
cm


Predicted  0.0  1.0  2.0  All
Actual                       
0.0         21    9    5   35
1.0          8   21    3   32
2.0          5    4   16   25
All         34   34   24   92